# Visualizing the data
In this notebook we will creae several visualization based on the curated metadata. <br>
The goal is to start providing context to the data base and investigate the distribution of fermented foods and thier microbiome. <br>
The data will not reflect true abundance of microbes in different foods/areas since it is based on presensce/absence counts from the metadata table. <br>

In [245]:
# make sure all libraries are installed before running this code

# importing standard libraries
import pandas as pd
import numpy as np
import time
from IPython.display import display
import json

# importing plotly libraries for dinamic visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import plotly.subplots as sp
import plotly.offline as pyo
import plotly.tools as tls
import plotly

# importing folium for better visualization using maps
import folium

# importing geolocation libraries so we can get the coordinates of the countries
import pycountry
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.extra.rate_limiter import RateLimiter



In [246]:
# loading the dataset
dataset = pd.read_csv('data/Food_MAGs_curated_metadata_250429.csv')

display(dataset.head(), dataset.shape)

,mag_id,sample_description,completeness,contamination,contigs,total_length,gc,n50,domain,phylum,class,order,family,genus,species,sample_accession,run_accession,country,project_accession,study_accession,database_origin,Reference,level1_substrate_origin,level2_main_ingredient_group,main_ingredient,food_name,consistency,main_ferment,acid_type,acid_level,alcohol_lovel,protein_degradation,fat_degradation,added_ingredients,fermentation_temp,aging_time,agin_temp,level3_food_type
0,C-03.Ssa-BR,raw canastra cheese,97.97,1.10,182,1896140,39.40,16848,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus sp003521145,SAMN17450155,SRR13496634,Brazil,PRJNA693797,SRP302686,NaN,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold,cheese
1,C-R02.bin.1,raw canastra cheese,92.75,1.03,84,3174852,65.79,74585,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Micrococcaceae,Galactobacter,Galactobacter sp.,SAMN17450161,SRR13496623,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold,cheese
2,C-R03.bin.7,raw canastra cheese,98.91,0.00,161,2047554,38.52,23316,Bacteria,Firmicutes,Bacilli,Lactobacillales,Aerococcaceae,Bavariicoccus,Bavariicoccus seileri,SAMN17450162,SRR13496622,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold,cheese
3,C-R06.bin.10,raw araxa cheese,91.62,1.35,147,3636195,37.73,38153,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus vulgaris,SAMN17450164,SRR13496620,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese araxa,solid,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold,cheese
4,C-R06.bin.12,raw araxa cheese,100.00,0.64,54,3940686,70.36,126465,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Dermabacteraceae,Brachybacterium,Brachybacterium alimentarium,SAMN17450164,SRR13496620,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese araxa,solid,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold,cheese


(13858, 38)

In [ ]:
# these corrections were made in the dataset before loading it
# # correcting several naming/spelling mistales I founf in the dataset

# # change aland islands fo Finland in country
# dataset['country'] = dataset['country'].replace({'Aland Islands': 'Finland'})
# # change boliva to Bolivia in country
# dataset['country'] = dataset['country'].replace({'Boliva': 'Bolivia'})
# # change Belgium:Antwerp to Belgium
# dataset['country'] = dataset['country'].replace({'Belgium:Antwerp': 'Belgium'})

# # correct 'cheese ' to 'cheese'
# dataset['level3_food_name'] = dataset['level3_food_name'].replace({'cheese ': 'cheese'})

# # correct choclate to chocolate
# dataset['level3_food_name'] = dataset['level3_food_name'].replace({'choclate': 'chocolate'})

# # correct saerkraut to sauerkraut
# dataset['level3_food_name'] = dataset['level3_food_name'].replace({'saerkraut': 'sauerkraut'})

# # correct 'cheese, curd' to 'cheese curd' for naming consistency
# dataset['level3_food_name'] = dataset['level3_food_name'].replace({'cheese, curd': 'cheese curd'})

# # correct 'cheese, hard' to 'hard cheese' for naming consistency
# dataset['level3_food_name'] = dataset['level3_food_name'].replace({'cheese, hard': 'hard cheese'})



In [ ]:
# saving corrected dataset
# dataset.to_csv('data/Food_MAGs_curated_metadata_250421_corrected_merged_final_v2_corrected.csv', index=False)

In [247]:
dataset.level3_food_type.nunique(),dataset.level3_food_type.unique()

(44,
 array(['cheese', 'fresh cheese', 'condiment', 'vinegar', 'cider',
        'pickled veggetables', 'beer', 'not fermented', 'starch', 'wine',
        'alcoholic cereal beverage', 'chocolate', 'coffee',
        'intermediate product for alcohol', 'cassava', 'dairy beverage',
        'seafood', 'intermediate for condiments',
        'alcoholic fruit beverage', 'bread', 'pickled fruit',
        'starter culture', 'alcoholic vegetable beverage', 'moromi',
        'sausage', 'Unknown', 'intermediate for wine', 'tofu whey',
        'shochu intermediate', 'whiskey intermediate',
        'fermented whole bean', 'nem chua', 'yogurt', nan, 'probiotics',
        'alcoholic dairy beverage', 'whey', 'porridge', 'gari',
        'cereal beverage', 'ugba', 'fura', 'massa', 'attieke', 'jalebi'],
       dtype=object))

In [248]:
# visualizing the number of samples (sample_accession) per country
# Get unique sample_accession and country pairs
unique_samples_by_country = dataset[['sample_accession', 'country']].drop_duplicates()

# # Count samples per country and sort in descending order
country_counts = unique_samples_by_country.value_counts('country').sort_values(ascending=False)
country_counts = country_counts.reset_index()
country_counts.columns = ['country', 'count']
# remove values that are lists
country_counts = country_counts[~country_counts['country'].str.contains('\[', na=False)]

# Create horizontal bar plot
fig = px.bar(country_counts, 
             y='country', 
             x='count',
             orientation='h',
             title="Number of unique samples per country",
             category_orders={'country': country_counts['country'].tolist()},
             height=800,
             width=800,)  

fig.update_layout(xaxis_title="Number of unique samples",
                 yaxis_title="Country",
                 yaxis=dict(tickfont=dict(size=8)))  # reduced font size for y-axis labels

fig.update_traces(marker_color='blue', marker_line_color='black',
                  marker_line_width=1.5, opacity=0.6)
fig.show()

In [249]:
# Define continent mapping
continent_map = {
    'Italy': 'Europe', 'Ireland': 'Europe', 'Spain': 'Europe', 'Austria': 'Europe', 
    'USA': 'North America', 'United Kingdom': 'Europe', 'Canada': 'North America',
    'Genrmany': 'Europe', 'France': 'Europe', 'China': 'Asia', 'Nigeria': 'Africa',
    'Australia': 'Oceania', 'Mexico': 'North America', 'Belgium': 'Europe',
    'Taiwan': 'Asia', 'India': 'Asia', 'Brazil': 'South America', 'Ghana': 'Africa',
    'Denmark': 'Europe', 'Singapore': 'Asia', 'Greece': 'Europe', 'Malaysia': 'Asia',
    'Norway': 'Europe', 'Turkey': 'Asia', 'Saudi Arabia': 'Asia', 
    'South Africa': 'Africa', 'Bulgaria': 'Europe', 'Russia': 'Europe',
    'Indonesia': 'Asia', 'Ecuador': 'South America', 'New Zealand': 'Oceania',
    'Finland': 'Europe', 'Korea': 'Asia', 'Colombia': 'South America',
    'Hong Kong': 'Asia', 'Estonia': 'Europe', 'hilippines': 'Asia',
    'Burkina Faso': 'Africa', 'Sweden': 'Europe', 'Tunisia': 'Africa',
    'Aland Islands': 'Europe', 'Croatia': 'Europe', 'Israel': 'Asia',
    'Lebanon': 'Asia', 'Portugal': 'Europe', 'Kenya': 'Africa',
    'Boliva': 'South America', 'Switzerland': 'Europe', 'Ethiopia': 'Africa',
    'Benin': 'Africa', 'Japan': 'Asia', 'Germany': 'Europe', 'Thailand': 'Asia',
    'Myanmar': 'Asia', 'Korean': 'Asia', 'Turkey ': 'Asia', 'Belgium:Antwerp': 'Europe'
}

# Add continent column
country_counts['continent'] = country_counts['country'].map(continent_map)

# Create scatter_geo plot with color by continent
fig = px.scatter_geo(country_counts,
                    locations='country',
                    locationmode='country names',
                    size='count',
                    color='continent',
                    hover_name='country',
                    hover_data={'count': True, 'continent': True},
                    projection='natural earth',
                    title='Number of Samples per Country')

fig.update_traces(marker=dict(opacity=0.6, line=dict(width=1, color='black')))

fig.update_layout(
    title_x=0.5,
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular',
        # Add zoom capabilities
        scope='world',
        showland=True,
        showcountries=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
        # Enable zoom
        visible=True
    ),
    width=1000,
    height=600
)

# Add buttons for zooming to different regions
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {'args': [{'geo.scope': 'world'}], 'label': 'World', 'method': 'relayout'},
            {'args': [{'geo.scope': 'europe'}], 'label': 'Europe', 'method': 'relayout'},
            {'args': [{'geo.scope': 'asia'}], 'label': 'Asia', 'method': 'relayout'},
            {'args': [{'geo.scope': 'africa'}], 'label': 'Africa', 'method': 'relayout'},
            {'args': [{'geo.scope': 'north america'}], 'label': 'North America', 'method': 'relayout'},
            {'args': [{'geo.scope': 'south america'}], 'label': 'South America', 'method': 'relayout'}
        ],
        'direction': 'down',
        'showactive': True,
        'x': 1.2,
        'y': 0.6  # Moved down from 0.9 to 0.75 to place below typical legend position
    }]
)

fig.show()

In [250]:

# Create base map centered roughly on world center
m = folium.Map(location=[20, 0], zoom_start=2)

# Add data points for each country
for index, row in country_counts.iterrows():
    # Get the country name and count
    country = row['country']
    count = row['count']
    
    # Add circle marker with size based on count
    # We need to get coordinates for each country, but since coordinates are not in the data
    # we'll need to use geocoding from previous cells (already set up)
    lat = None
    lon = None
    
    try:
        location = geocode(country)
        if location:
            lat = location.latitude
            lon = location.longitude
    except:
        continue
        
    if lat and lon:
        # Create circle marker with radius proportional to count
        folium.CircleMarker(
                    location=[lat, lon],
                    radius=5 + (count / country_counts['count'].max() * 25),  # Scale between 5-30
                    popup=f"{country}: {count} samples",
                    color='blue',
                    fill=True,
                    fill_color='blue',
                    fill_opacity=0.6
                ).add_to(m)

# Display the map
display(m)


In [252]:
# visualizing the number of samples (sample_accession) per country
# Get unique sample_accession and country pairs
unique_food_by_country = dataset[['sample_accession', 'country','level3_food_type']].drop_duplicates()

# Count samples per country and sort in descending order
food_counts = unique_food_by_country.groupby(['country', 'level3_food_type']).size().reset_index(name='Count')
food_counts = food_counts.sort_values(['country','Count'], ascending=[False, True])


In [254]:
# correcting for rows that have multiple countries in the same cell
# First, create a copy of the dataframe
new_food_counts = food_counts.copy()

# Find rows where country contains '[' 
mask = new_food_counts['country'].str.contains('\[', na=False)
rows_to_expand = new_food_counts[mask].copy()

# Remove these rows from the original dataframe
new_food_counts = new_food_counts[~mask]

# Process rows that need to be expanded
for idx, row in rows_to_expand.iterrows():
    # Extract country names from the string
    countries = eval(row['country'])
    
    # Create new rows for each country
    for country in countries:
        new_row = pd.DataFrame({
            'country': [country],
            'level3_food_type': [row['level3_food_type']],
            'Count': [row['Count']]
        })
        new_food_counts = pd.concat([new_food_counts, new_row], ignore_index=True)

# Sort the resulting dataframe
new_food_counts = new_food_counts.sort_values(['country', 'Count'], ascending=[True, False])

# Update food_counts with the new dataframe
food_counts = new_food_counts.copy()

In [255]:
 # change korean to Korea
food_counts['country'] = food_counts['country'].replace({'korean': 'Korea'})

In [256]:
# add a column with the % of a food items per country
food_counts['percentage'] = food_counts.groupby('country')['Count'].transform(lambda x: x / x.sum() * 100).round(2)



In [257]:
# change the column name level3_food_name to food
food_counts = food_counts.rename(columns={'level3_food_type': 'food'})


In [258]:
pd.set_option('display.max_rows', None)
display(food_counts.groupby('food').Count.sum().sort_values(ascending=False))

food
cheese                          1505
alcoholic fruit beverage         389
dairy beverage                   291
sausage                          131
bread                             76
whey                              71
pickled veggetables               64
condiment                         51
alcoholic vegetable beverage      35
chocolate                         35
alcoholic dairy beverage          33
wine                              27
yogurt                            22
alcoholic cereal beverage         18
cereal beverage                   16
fermented whole bean              15
probiotics                        12
porridge                          10
beer                               9
fresh cheese                       8
coffee                             7
seafood                            5
gari                               5
pickled fruit                      5
starch                             3
ugba                               3
jalebi                           

In [259]:
food_counts.Count.sum()

np.int64(2856)

In [261]:
# Create a list of unique foods to determine how many colors we need
unique_foods = food_counts['food'].unique()

# Create a color map using distinct colors from plotly express
colors = px.colors.qualitative.Set3 + px.colors.qualitative.Pastel + px.colors.qualitative.Safe

# Create color dictionary
color_map = dict(zip(unique_foods, colors[:len(unique_foods)]))

fig1 = px.bar(food_counts,
             y='country',
             x='percentage',
             color='food',
             color_discrete_map=color_map,  # Use our custom color map
             orientation='h',
             title="Percentage of researched food per country",
             category_orders={'country': food_counts['country'].tolist()},
             height=1000,
             width=800)

# Update layout
fig1.update_layout(
    showlegend=True,
    legend_title_text="Food Types",
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    ),
    margin=dict(r=200)
)

fig1.show()

In [262]:
import pycountry
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
from geopy.extra.rate_limiter import RateLimiter

# Initialize geolocator with increased timeout
geolocator = Nominatim(user_agent="country_locator", timeout=10)

# Create a rate-limited version of the geocoding function
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to get coordinates with retry mechanism
def get_coordinates(country_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Manual mapping for special cases
            country_mapping = {
                'USA': 'United States',
                'Korea': 'South Korea',
                'hilippines': 'Philippines',
                'Genrmany': 'Germany'
            }
            
            search_name = country_mapping.get(country_name, country_name)
            
            try:
                # Try using pycountry first
                country = pycountry.countries.lookup(search_name)
                search_name = country.name
            except (LookupError, AttributeError):
                # If pycountry fails, use the original or mapped name
                pass
            
            # Add explicit country type to search
            location = geocode(f"{search_name}", featuretype='country')
            if location:
                return pd.Series([location.latitude, location.longitude])
            
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            if attempt == max_retries - 1:
                print(f"Failed to geocode {country_name} after {max_retries} attempts: {str(e)}")
            else:
                time.sleep(2 ** attempt)
                continue
        except Exception as e:
            print(f"Error processing {country_name}: {str(e)}")
            break
            
    return pd.Series([None, None])

# Process unique countries
unique_countries = food_counts['country'].unique()
coordinates_df = pd.DataFrame(unique_countries, columns=['country'])
coordinates_df[['latitude', 'longitude']] = coordinates_df['country'].apply(get_coordinates)

# Merge coordinates back to the main DataFrame
food_counts = food_counts.merge(coordinates_df, on='country', how='left')


In [263]:
food_counts[food_counts['latitude'].isna()]

,country,food,Count,percentage,latitude,longitude
63,Hong Kong,alcoholic fruit beverage,5,100.0,NaN,NaN
155,Taiwan,dairy beverage,28,100.0,NaN,NaN


In [264]:
# find the lat and lon for Taiwan
taiwan = geocode('Taiwan')
# add the lat and lon for Taiwan to the food_counts dataframe
food_counts.loc[food_counts['country'] == 'Taiwan', 'latitude'] = taiwan.latitude
food_counts.loc[food_counts['country'] == 'Taiwan', 'longitude'] = taiwan.longitude

#find the lat and lon for hong kong
hong_kong = geocode('Hong Kong')
# add the lat and lon for Hong Kong to the food_counts dataframe
food_counts.loc[food_counts['country'] == 'Hong Kong', 'latitude'] = hong_kong.latitude
food_counts.loc[food_counts['country'] == 'Hong Kong', 'longitude'] = hong_kong.longitude

In [266]:
# Create the base map with a lighter color scheme
m = folium.Map(location=[20, 0], zoom_start=2, tiles='CartoDB positron')

for country in food_counts['country'].unique():
    country_data = food_counts[food_counts['country'] == country]
    lat = country_data['latitude'].values[0]
    lon = country_data['longitude'].values[0]
    
    if pd.notnull(lat) and pd.notnull(lon):
        # Create a copy of country_data
        plot_data = country_data.copy()
        
        # Identify foods with percentage < 1%
        small_foods = plot_data[plot_data['percentage'] < 1]
        
        # Remove small foods from plot_data
        plot_data = plot_data[plot_data['percentage'] >= 1]
        
        # Add 'Other' row if there are small foods
        if not small_foods.empty:
            other_row = pd.DataFrame({
                'country': [country],
                'food': ['Other'],
                'Count': [small_foods['Count'].sum()],
                'percentage': [small_foods['percentage'].sum()],
                'latitude': [lat],
                'longitude': [lon]
            })
            plot_data = pd.concat([plot_data, other_row])
        
        # Create pie chart using plotly
        fig = px.pie(plot_data, 
                 values='percentage',
                 names='food',
                 title=country,
                 hover_data=['Count'])
        
        # Update layout with smaller dimensions and better legend placement
        fig.update_layout(
            width=300,
            height=300,
            margin=dict(l=20, r=120, t=40, b=20),  # Increased right margin for legend
            showlegend=True,
            legend=dict(
            orientation="v",
            yanchor="middle",
            y=0.5,
            xanchor="left", 
            x=1.2,  # Moved legend further right
            font=dict(size=8)  # Smaller legend font
            )
        )
        
        # Convert plotly figure to HTML
        html_str = fig.to_html(
            include_plotlyjs='cdn',
            full_html=False,
            config={'displayModeBar': False}
        )
        
        # Create iframe and popup
        iframe = folium.IFrame(html=html_str, width=350, height=350)
        popup = folium.Popup(iframe, max_width=400)
        
        # Add marker
        folium.CircleMarker(
            location=[lat, lon],
            radius=8,
            color='blue',
            fill=True,
            popup=popup
        ).add_to(m)

# Display the map
display(m)

In [236]:
pd.set_option('display.max_columns', None)
dataset.head()

,mag_id,sample_description,fermented_food,specific_substrate,substrate_category,general_category,completeness,contamination,contigs,total_length,gc,n50,domain,phylum,class,order,family,genus,species,study_catalog,source,split,sample_accession,run_accession,subtype,country,project_accession,study_accession,database_origin,Reference,level1_substrate_origin,level2_main_ingredient_group,main_ingredient,level3_food_name,consistency,main_ferment,acid_type,acid_level,alcohol_lovel,protein_degradation,fat_degradation,added_ingredients,fermentation_temp,aging_time,agin_temp
0,C-03.Ssa-BR,raw canastra cheese,cheese,milk,dairy,fermented_dairy,97.97,1.10,182,1896140,39.40,16848,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus sp003521145,Caffrey2024,Kothe et al. 2021,Caffrey2024,SAMN17450155,SRR13496634,NaN,Brazil,PRJNA693797,SRP302686,NaN,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold
1,C-R02.bin.1,raw canastra cheese,cheese,milk,dairy,fermented_dairy,92.75,1.03,84,3174852,65.79,74585,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Micrococcaceae,Galactobacter,Galactobacter sp.,Caffrey2024,Kothe et al. 2021,Caffrey2024,SAMN17450161,SRR13496623,NaN,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold
2,C-R03.bin.7,raw canastra cheese,cheese,milk,dairy,fermented_dairy,98.91,0.00,161,2047554,38.52,23316,Bacteria,Firmicutes,Bacilli,Lactobacillales,Aerococcaceae,Bavariicoccus,Bavariicoccus seileri,Caffrey2024,Kothe et al. 2021,Caffrey2024,SAMN17450162,SRR13496622,NaN,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold
3,C-R06.bin.10,raw araxa cheese,cheese,milk,dairy,fermented_dairy,91.62,1.35,147,3636195,37.73,38153,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus vulgaris,Caffrey2024,Kothe et al. 2021,Caffrey2024,SAMN17450164,SRR13496620,NaN,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese araxa,solid,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold
4,C-R06.bin.12,raw araxa cheese,cheese,milk,dairy,fermented_dairy,100.00,0.64,54,3940686,70.36,126465,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Dermabacteraceae,Brachybacterium,Brachybacterium alimentarium,Caffrey2024,Kothe et al. 2021,Caffrey2024,SAMN17450164,SRR13496620,NaN,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese araxa,solid,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold


In [267]:
# visualizing the number of samples (sample_accession) per food
# Get unique sample_accession and country pairs
unique_samples_by_food = dataset[['sample_accession', 'level3_food_type']].drop_duplicates()

# # Count samples per country and sort in descending order
food_counts = unique_samples_by_food.value_counts('level3_food_type').sort_values(ascending=False)
food_counts = food_counts.reset_index()
food_counts.columns = ['Food', 'count']
# remove values that are lists
food_counts = food_counts[~food_counts['Food'].str.contains('\[', na=False)]

# Create horizontal bar plot
fig = px.bar(food_counts, 
             y='Food', 
             x='count',
             orientation='h',
             title="Number of unique samples per food",
             category_orders={'Food': food_counts['Food'].tolist()},
             height=2000,
             width=800,)  

fig.update_layout(xaxis_title="Number of unique samples",
                 yaxis_title="Food",
                 yaxis=dict(tickfont=dict(size=8)))  # reduced font size for y-axis labels

fig.update_traces(marker_color='blue', marker_line_color='black',
                  marker_line_width=1.5, opacity=0.6)
fig.show()

In [268]:
dataset.head()

,mag_id,sample_description,completeness,contamination,contigs,total_length,gc,n50,domain,phylum,class,order,family,genus,species,sample_accession,run_accession,country,project_accession,study_accession,database_origin,Reference,level1_substrate_origin,level2_main_ingredient_group,main_ingredient,food_name,consistency,main_ferment,acid_type,acid_level,alcohol_lovel,protein_degradation,fat_degradation,added_ingredients,fermentation_temp,aging_time,agin_temp,level3_food_type
0,C-03.Ssa-BR,raw canastra cheese,97.97,1.10,182,1896140,39.40,16848,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus sp003521145,SAMN17450155,SRR13496634,Brazil,PRJNA693797,SRP302686,NaN,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold,cheese
1,C-R02.bin.1,raw canastra cheese,92.75,1.03,84,3174852,65.79,74585,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Micrococcaceae,Galactobacter,Galactobacter sp.,SAMN17450161,SRR13496623,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold,cheese
2,C-R03.bin.7,raw canastra cheese,98.91,0.00,161,2047554,38.52,23316,Bacteria,Firmicutes,Bacilli,Lactobacillales,Aerococcaceae,Bavariicoccus,Bavariicoccus seileri,SAMN17450162,SRR13496622,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese canastra,solid,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold,cheese
3,C-R06.bin.10,raw araxa cheese,91.62,1.35,147,3636195,37.73,38153,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Proteus,Proteus vulgaris,SAMN17450164,SRR13496620,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese araxa,solid,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold,cheese
4,C-R06.bin.12,raw araxa cheese,100.00,0.64,54,3940686,70.36,126465,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Dermabacteraceae,Brachybacterium,Brachybacterium alimentarium,SAMN17450164,SRR13496620,Brazil,PRJNA693797,SRP302686,NCBI,"Kothe CI, Mohellibi N, Renault P. Revealing th...",Animal,Dairy,milk,cheese araxa,solid,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold,cheese


In [293]:
# subsetting genus, sample_accession, level3_food_type
dataset_family = dataset[['family', 'level3_food_type']]
# calculating genus numbers per sample_accession and level3_food_type
family_counts = dataset_family.groupby(['family', 'level3_food_type'])['family'].count().reset_index(name='Count')
# rename level3_food_type to food
family_counts = family_counts.rename(columns={'level3_food_type': 'food'})
# calculating the percentage of each family per food
family_counts['percentage'] = family_counts.groupby('food')['Count'].transform(lambda x: x / x.sum() * 100).round(2)

In [294]:
family_counts.head()

,family,food,Count,percentage
0,Acetobacteraceae,Unknown,1,50.00
1,Acetobacteraceae,alcoholic cereal beverage,9,5.62
2,Acetobacteraceae,alcoholic dairy beverage,11,7.91
3,Acetobacteraceae,alcoholic fruit beverage,352,19.06
4,Acetobacteraceae,alcoholic vegetable beverage,61,31.44


In [297]:
# making a bar chart of family in food using family_counts and faceting by level3_food_typepe bar chart of family in food using family_counts and faceting by level3_food_type
fig = px.bar(family_counts, 
             x='family', 
             y='percentage',
             facet_col='food',  # Add faceting by food type
             facet_col_wrap=5,  # Adjust the number of columns for facets
             title="Percent of microbes by family and food type",
             color='family',
             height=3000,  # Increased height to accommodate facets
             width=2000)

# Update layout
fig.update_layout(
    showlegend=False,
    yaxis_title="Percent",
    xaxis_title="family"
)

# Update facet layout to not share axes
fig.update_xaxes(showticklabels=True, matches=None, tickangle=90)
#fig.update_yaxes(showticklabels=True, matches=None)

# Update font sizes for better readability
fig.update_annotations(font_size=10)
fig.update_xaxes(tickfont_size=8)

# Show the plot
fig.show()